In [1]:
# our nice list of imports is in here
#i hate this, but the notebook won't recognize our custom modules w/o it
import sys
sys.path.append("..")
import lib
from lib import utils

import numpy as np
import pandas as pd
import os
import urllib
import random
from shutil import copyfile
import matplotlib.pyplot as plt
import cv2
import matplotlib.patches as patches

import tensorflow as tf
import keras
from keras.utils import to_categorical
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential        # keras is only compatible with python 3.6 or lower
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


from skimage import io, data
from skimage.util import img_as_ubyte
from skimage.transform import rescale

import sklearn
from sklearn.preprocessing import LabelEncoder

from mtcnn.mtcnn import MTCNN

from IPython.display import clear_output

import seaborn as sns
%matplotlib inline
sns.set_style("darkgrid")

Using TensorFlow backend.


# Face Mask Detection


### Brief Overview

Simply put, our goal is to create a neural network that will (1) identify face masks or lack thereof on a human face, and (2) determine whether that facemask is correctly worn.


Luckily for us, Kaggle had a contest earlier this year related to (1) -- so a dataset was readily available, and it has proven to be quite effective at training a mask/no-mask neural net with our testing images. (Kaggle)


However, the dataset is extremely lacking when it comes to incorrect mask usage.  We did locate an additional dataset that consists only of faces with correctly worn masks and incorrectly worn masks. (CMFD/IMFD)

### Data

1. Training Data - We have training data from both datasets.  The images in each set have been pre-processed with relevant data written to CSV files to ease the process load.
  + Kaggle Dataset -- 
2. Testing Data - Our testing data consists of 


### Models

To that end, we need a few different models:
1. One to predict whether there's a mask on a face or no mask
  + For this, we will use the Kaggle Dataset, and focus on the two largest features available in it: whether there is a face mask or not.
  + Potentially, we could add the data from the CMFD/IMFD dataset as representations of masks for the training set, since we'll be determining whether they're correct or incorrectly worn in the next stage.  The only concern there is a dataset that's very, very heavy on masks vs. no masks
2. One to predict whether, given a mask is on a face, it is worn correctly
  + For this, we will use the CMFD/IMFD dataset.  There are only two features to this dataset: correctly worn and incorrectly worn face masks.




In [2]:
# define directory paths for easier navigation
NOTEBOOK_DIR = !pwd
NOTEBOOK_DIR = NOTEBOOK_DIR[0]
ROOT_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir))
DATA_DIR = os.path.join(ROOT_DIR, 'data')
IMAGES_DIR = os.path.join(DATA_DIR, 'images')
IMG_DATA_DIR = os.path.join(DATA_DIR, 'img_data')
KAGGLE_DIR = os.path.join(IMAGES_DIR, 'kaggle')
HC_DATA_DIR = os.path.join(DATA_DIR, 'haar_cascades')

# MNM_TRIAL_DIR = os.path.join(IMAGES_DIR, 'mnm_trial')
# MNM_TRAINING = os.path.join(MNM_TRIAL_DIR, 'training')
# MNM_TESTING = os.path.join(MNM_TRIAL_DIR, 'testing')

PRAJNA_SRC_DIR = os.path.join(DATA_DIR, 'prajna-dataset')
PMASK_SRC = os.path.join(PRAJNA_SRC_DIR, 'mask')
PNOMASK_SRC = os.path.join(PRAJNA_SRC_DIR, 'nomask')

PTRAINING_DIR = os.path.join(PRAJNA_SRC_DIR, 'dest/train')
PTESTING_DIR = os.path.join(PRAJNA_SRC_DIR, 'dest/test')
PVAL_DIR = os.path.join(PRAJNA_SRC_DIR, 'dest/val')

PTRAINING_MASK = os.path.join(PRAJNA_SRC_DIR, 'dest/train/mask')
PTESTING_MASK = os.path.join(PRAJNA_SRC_DIR, 'dest/test/mask')
PVAL_MASK = os.path.join(PRAJNA_SRC_DIR, 'dest/val/mask')

PTRAINING_NOMASK = os.path.join(PRAJNA_SRC_DIR, 'dest/train/nomask')
PTESTING_NOMASK = os.path.join(PRAJNA_SRC_DIR, 'dest/test/nomask')
PVAL_NOMASK = os.path.join(PRAJNA_SRC_DIR, 'dest/val/nomask')



# MASK_SRC_DIR=os.path.join(MNM_TRIAL_DIR, 'mask')
# TRAINING_MASK=os.path.join(MNM_TRAINING, 'mask')
# TESTING_MASK=os.path.join(MNM_TESTING, 'mask')

# NOMASK_SRC_DIR=os.path.join(MNM_TRIAL_DIR, 'nomask')
# TRAINING_NOMASK=os.path.join(MNM_TRAINING, 'nomask')
# TESTING_NOMASK=os.path.join(MNM_TESTING, 'nomask')

KAGGLE_IMAGES_DIR = "https://github.com/brtonnies/face-mask-detection/blob/data-branch/data/images"
# CMFD_IMAGES_DIR = "https://github.com/brtonnies/face-mask-detection/blob/data-branch/data/images/CMFD/images"
# IMFD_IMAGES_DIR = "https://github.com/brtonnies/face-mask-detection/blob/data-branch/data/images/IMFD/images"

# Mask vs. No Mask (MNM)

## Model Preparations

+ We keep pre-compiled versions of our different models on hand (again, to reduce process/load times)
+ Unless the training data changes, we default to using those existing models

1. First model used ONLY Prajna Dataset (97% acc)
2. Second model is training now, with additional 'no mask' images from Kaggle
3. Third will have additional 'mask' images from kaggle

**DON'T FORGET TO REVERT IMAGES TO BEST MODEL VERSION**

In [3]:
def split_data(src, train, test, split_size):
    dataset = []
    
    for unitData in os.listdir(src):
        data = os.path.join(src, unitData)
        if(os.path.getsize(data) > 0 and os.path.isfile(data)):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * split_size)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = os.path.join(src, unitData)
        final_train_set = os.path.join(train, unitData)
        
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = os.path.join(src, unitData)
        final_test_set = os.path.join(test, unitData)
        copyfile(temp_test_set, final_test_set)

        
split_size = 0.8
split_data(PMASK_SRC, PTRAINING_MASK, PTESTING_MASK, split_size)
split_data(PNOMASK_SRC, PTRAINING_NOMASK, PTESTING_NOMASK, split_size)


## Keras Sequential Model

#### Building the Model
In this proposed method, the Face Mask detection model is built using the Sequential API of the keras library. This allows us to create the new layers for our model step by step. The various layers used for our CNN model is described below.

1. The first layer is Conv2D with **100 kernels** of size $3\times3$. 
  + The activation function is ‘ReLu’.  
  + The input size is set to $150\times 150 \times 3$ for all the images to be trained and tested

2. MaxPooling2D is used for the second layer with pool size of $2 \times 2$.

3. The next layer is another Conv2D layer 
  + Has its own 100 kernels of the same size: $3\times3$ 
  + Activation function is ‘ReLu’. 
  + The layer is followed by another MaxPooling2D layer with pool size $2 \times 2$.

4. Then the Flatten() layer to flatten *all* the layers into a single dimension.

5. **NEW:** After the Flatten layer, we added dropout (0.5) layer to prevent the model from overfitting -- which was killing us in early trials.

6. Then we used the Dense layer 
  + 50 'units' 
  + Activation function: ‘ReLu’.

7. The last layer of our model will be another Dense Layer, with only two units and activation function ‘Softmax’. The softmax function outputs a vector representing the probability distributions of each of the input units. Here, two input units are used. The softmax function will output a vector with two probability distribution values: is there a mask on this face or not?

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

kernel_size = (3, 3)
image_shape = (150, 150, 3)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, kernel_size, activation='relu', input_shape=image_shape, strides=2),
    tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(32, (3,3), activation='tanh'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, kernel_size, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),   # previously we were way overfitting, so we added dropout
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax'),
])

opt = tf.keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-5)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [5]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(PTRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))

validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(PTESTING_DIR, 
                                                              batch_size=10, 
                                                              target_size=(150, 150))

# (1) train the all data extended model
# (2) re-train prajna, prajna-kaggle-mask to save new checkpoints (and load them :D)
# (3) also try and do one model where only new masks are added to the set

model_path = '../models/mnm/prajna'
checkpoint_path = '../models/mnm/prajna.checkpoint' 

# model_path = '../models/mnm/prajna-kaggle-nomask-extend'
# checkpoint_path = '../models/mnm/prajna-kaggle-nomask-extend-{epoch:03d}.model'
# model_path = '../models/mnm/prajna-kaggle-mask-extend'
# checkpoint_path = '../models/mnm/prajna-kaggle-mask-extend-{epoch:03d}.model'
# model_path = '../models/mnm/prajna-kaggle-all-extend'
# checkpoint_path = '../models/mnm/prajna-kaggle-all-extend-{epoch:03d}.model'


# NOTE: here's something cool - we can use checkpoints
#       to save/load the weights from previous model training
# if os.path.exists(checkpoint_path):
#     model.load_weights(os.path.join(checkpoint_path, 'variables/variables'))
    
# alternatively, we can load the whole model from the checkpoint state
# if os.path.exist(checkpoint_path):
#     model = load_model(checkpoint_path_path)

checkpoint = ModelCheckpoint(
    checkpoint_path, 
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True,
    mode = 'auto'
)


Found 1105 images belonging to 2 classes.
Found 278 images belonging to 2 classes.


In [6]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/30
110/111 [============================>.] - ETA: 0s - loss: 0.5191 - acc: 0.7388
Epoch 00001: val_loss improved from inf to 0.22289, saving model to ../models/mnm/prajna.checkpoint
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../models/mnm/prajna.checkpoint/assets
111/111 [==============================] - 55s 494ms/step - loss: 0.5216 - acc: 0.7376 - val_loss: 0.2229 - val_acc: 0.9173
Epoch 2/30
110/111 [============================>.] - ETA: 0s - loss: 0.3337 - acc: 0.8731
Epoch 00002: val_loss improved from 0.22289 to 0.12734, saving model to ../models/mnm/prajna.checkpoint
INFO:tensorflow:Assets written to: ../models/mnm/prajna.checkpoint/assets
111/111 [==============================] - 53s 476ms/step - loss: 0.3340 - acc: 0.8724 - val_loss: 0.1273 - val_acc: 0.9568
Epoch 3/30
110/111 [============================>.] - ETA: 0s - loss: 0.2809 - acc: 0.8877
Epoch 00003: val_loss improved from 0.12734 to

Epoch 26/30
110/111 [============================>.] - ETA: 0s - loss: 0.1631 - acc: 0.9406
Epoch 00026: val_loss did not improve from 0.01202
111/111 [==============================] - 55s 499ms/step - loss: 0.1632 - acc: 0.9403 - val_loss: 0.0758 - val_acc: 0.9496
Epoch 27/30
110/111 [============================>.] - ETA: 0s - loss: 0.0835 - acc: 0.9744
Epoch 00027: val_loss did not improve from 0.01202
111/111 [==============================] - 57s 516ms/step - loss: 0.0829 - acc: 0.9747 - val_loss: 0.0128 - val_acc: 0.9964
Epoch 28/30
110/111 [============================>.] - ETA: 0s - loss: 0.0796 - acc: 0.9744
Epoch 00028: val_loss improved from 0.01202 to 0.00946, saving model to ../models/mnm/prajna.checkpoint
INFO:tensorflow:Assets written to: ../models/mnm/prajna.checkpoint/assets
111/111 [==============================] - 60s 542ms/step - loss: 0.0793 - acc: 0.9747 - val_loss: 0.0095 - val_acc: 0.9964
Epoch 29/30
110/111 [============================>.] - ETA: 0s - loss: 0

# How Did We Do?

To answer that, we have to answer a few other questions:
1. What do the training/validation results look like?
2. How well does our model predict a completely different dataset?
  + For this, we'll use the training set from Kaggle so we can also check our accuracy
3. How well does our model predict in a different setting?
  + We'll use a live WebCam feed or video file and have our model predict in real time (or close to it -- framerate is spotty)

## Predicting Another Dataset

In [78]:
# let's see how well it predicts on another dataset
df = pd.read_csv(os.path.join(IMG_DATA_DIR, 'kaggle_training.csv'), index_col=0)

# the kaggle dataset actually had errors in its labeling -- on purpose
df.rename(columns={'x1': 'x', 'x2': 'y', 'y1': 'w', 'y2': 'h'}, inplace=True)
df.drop(['bbox'], axis=1, inplace=True)

df['mask'] = np.where(df['classname'].isin(
    ['face_with_mask', 'mask_surgical', 'mask_colorful']),
    1, 0
)
                      
test_images = list(df['name'])
df

,name,x,y,w,h,classname,mask
0,2756.png,69,126,294,392,face_with_mask,1
1,2756.png,505,10,723,283,face_with_mask,1
2,2756.png,75,252,264,390,mask_colorful,1
3,2756.png,521,136,711,277,mask_colorful,1
4,6098.jpg,360,85,728,653,face_no_mask,0
...,...,...,...,...,...,...,...
15407,1894.jpg,437,121,907,644,face_with_mask,1
15408,1894.jpg,557,363,876,636,mask_surgical,1
15409,1894.jpg,411,3,940,325,hat,0
15410,3216.png,126,69,409,463,face_with_mask,1


In [116]:
labels_dict={ 0:'No Mask', 1:'Mask' }
color_dict={ 0: (0,0,255), 1: (0,255,0) }

name = test_images[0]

# fetch the image from the remote repository
url = os.path.join(KAGGLE_IMAGES_DIR, "{}?raw=true".format(name))
resp = urllib.request.urlopen(url)
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)


# image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#     print(image[0, 0])

# locate the image within the dataframe so we can find bounding boxes
# NOTE: bounding boxes were pre-detected in this dataset and stored in the dataframe CSV
to_test = df[df['name'] == name]
#     print(to_test)


# print([i for i in list(faces) if len(i) > 4])
for i in range(len(to_test)):
    im = df.iloc[i]
#     print(im['x'])

    face_region = image[im['y']:im['y']+im['h'], im['x']:im['x']+im['w']]
#     print(face_region.shape)
    resized = cv2.resize(face_region, (150,150))
    normalized = resized/255.0
    reshaped = np.reshape(normalized, (1, 150, 150, 3))
    reshaped = np.vstack([reshaped])
    pred = model.predict(face_region)
    print(pred)
        
plt.imshow(image, cmap='gray')

ValueError: cannot reshape array of size 22500 into shape (1,150,150,3)

In [42]:
cv2_image = img_as_ubyte(image[:, :, ::-1])
gray = cv2.cvtColor(cv2_image, cv2.IMREAD_GRAYSCALE)
plt.imshow(gray)

a = [69, 126, 294, 392]
x, y, w, h = a
len(a)


IndexError: too many indices for array

In [30]:

for name in test_images:
    # fetch the image from the remote repository
    url = os.path.join(KAGGLE_IMAGES_DIR, "{}?raw=true".format(name))
    image = io.imread(url)
    
    # convert the RGB image into BGR then Grayscale for compatibility with OpenCV
    image = img_as_ubyte(image[:, :, ::-1])
    image = cv2.cvtColor(image, cv2.IMREAD_GRAYSCALE)
    
#     print(image[0, 0])
    
    # locate the image within the dataframe so we can find bounding boxes
    # NOTE: bounding boxes were pre-detected in this dataset and stored in the dataframe CSV
    to_test = df[df['name'] == name]
#     print(to_test)
        
    faces = to_test['bbox']
#     print(faces)
    for face in list(faces):
#         print("\n{}\n".format(face))
#         if len(face) == 4 and image is not None:
            print(face)
            x, y, w, h = face
            face_region = image[y:y+h, x:x+w]
            resized = cv2.resize(face, (150,150))
            normalized = resized/255.0
            reshaped = np.reshape(normalized, (1, 150, 150, 3))
            reshaped = np.vstack([reshaped])
            pred = model.predict(reshaped)
            print(pred)
        

        
test_images    

KeyboardInterrupt: 

## Using Our Model With Video Feed

+ We have two available facial detection algorithms: 
  1. Haar Cascade - faster detection, but only tuned for full, frontal views of faces.
  2. MTCNN - slower detection, but returns extra features and can detect faces from a variety of angles -- not only the front.


In [14]:
# use haar cascade to detect faces (frontal)
HC_FRONTAL_FACE = os.path.join(HC_DATA_DIR, 
                               'haarcascade_frontalface_default.xml')
classifier = cv2.CascadeClassifier(HC_FRONTAL_FACE)


# use MTCNN to detect faces
# classifier = MTCNN()


labels_dict={ 0:'No Mask', 1:'Mask' }
color_dict={ 0: (0,0,255), 1: (0,255,0) }

size = 4

# open webcam feed
# cap = cv2.VideoCapture(0)

# alternatively, open feed from a file
cap = cv2.VideoCapture('/Users/brtonnies/ArtificialIntelligence/comporg.mov')

# EVEN from a mobile device (UNTESTED)
# cap = cv.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if frame is not None:
        # frame = cv2.flip(frame, 1, 1) # flip so view looks more natural live (?)
        

        # detection is a little faster on smaller images -- so let's make it small
        small = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))

        # detect faces with haar cascades
        faces = classifier.detectMultiScale(small)
        
        # detect faces with MTCNN
#         faces = classifier.detect_faces(small)

        for f in faces:
            # reverse the scale-down of the frame for bounding box (haar)
            (x, y, w, h) = [v * size for v in f] 
            keypoints = None
            
            # reverse the scale-down of the frame for bounding box (MTCNN)
#             (x, y, w, h) = [v * size for v in f['box']]
            # print(f['keypoints'])
#             keypoints = f['keypoints']
                
        
            face = frame[y:y+h, x:x+w]
            resized = cv2.resize(face, (150,150))
            normalized = resized/255.0
            reshaped = np.reshape(normalized,(1,150,150,3))
            reshaped = np.vstack([reshaped])
            result = model.predict(reshaped)
#             print(result)

            label = np.argmax(result, axis=1)[0]
#             print("PREDICTION: {}".format(labels_dict[label]))
            cv2.rectangle(frame, (x,y), (x+w,y+h), color_dict[label], 2)
            cv2.rectangle(frame, (x,y-40), (x+w,y), color_dict[label], -1)
            cv2.putText(
                frame, 
                labels_dict[label], 
                (x, y-10), 
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (255,255,255),
                2
            )
            
            # extra fun when using MTCNN :)
            if keypoints is not None:
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['left_eye']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['right_eye']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['nose']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['mouth_left']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['mouth_right']])), 
                    2, (255,0,255), 2)
            # end of extra fun with MTCNN

        cv2.imshow('LIVE DETECTION ACTIVE', frame)
        key = cv2.waitKey(10)

        if key == 27: # if Esc key pressed -- end feed
            break
            
    else:
        break
        
# stop video
cap.release()

# close windows
cv2.destroyAllWindows()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
cap.release()
cv2.destroyAllWindows()